## Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path

# check if workding_dir is in local variables
if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)

os.chdir(workding_dir)
sys.path.append(workding_dir)
# print("workding dir:", workding_dir)

## Benchmark-LangGraph-Validation

In [3]:
import pandas as pd

df = pd.read_excel("./dataset/expert-annotations.xlsx", skiprows=[0, 102, 103, 104, 105])
df

,Ticket ID,Title/Summary,Method,Valid Output?,Gherkin Compliant?,Relevant to Ticket?,Quality/Coverage?,Comment,Unnamed: 8
0,1,Implement/Test API: GET /1.0/kb/accounts/{acco...,GET,YES,YES,YES,9,"Has +ve/-ve cases, validation, authorization, ...",NaN
1,2,Implement/Test API: POST /1.0/kb/accounts/{acc...,POST,YES,YES,YES,8,"Has +ve/-ve cases, validation, authorization, ...",NaN
2,3,Implement/Test API: DELETE /1.0/kb/accounts/{a...,DELETE,YES,YES,YES,8,"Has +ve/-ve cases, validation, authorization, ...",NaN
3,4,Implement/Test API: GET /1.0/kb/accounts/{acco...,GET,YES,YES,YES,9,"Has +ve/-ve cases, validation, authorization, ...",NaN
4,5,Implement/Test API: POST /1.0/kb/accounts/{acc...,POST,YES,YES,YES,8,"Has +ve/-ve cases, validation, authorization, ...",NaN
...,...,...,...,...,...,...,...,...,...
95,96,Implement/Test API: DELETE /1.0/kb/invoicePaym...,DELETE,YES,YES,YES,9,"Has +ve/-ve cases, validation, authorization, ...",NaN
96,97,Implement/Test API: GET /1.0/kb/invoicePayment...,GET,YES,YES,YES,8,"Has +ve/-ve cases, validation, authorization, ...",NaN
97,98,Implement/Test API: POST /1.0/kb/invoicePaymen...,POST,YES,YES,YES,9,"Has +ve/-ve cases, validation, authorization, ...",NaN
98,99,Implement/Test API: PUT /1.0/kb/invoicePayment...,PUT,YES,YES,YES,9,"Has +ve/-ve cases, validation, authorization, ...",NaN


In [4]:
df2 = pd.read_csv("./dataset/synthetic-jira-tickets-expanded.csv")
df2

,id,method,valid,title,description
0,1,GET,True,Implement/Test API: GET /1.0/kb/accounts/{acco...,This API retrieves tags for a specific account...
1,2,POST,True,Implement/Test API: POST /1.0/kb/accounts/{acc...,This API adds tags to a specific account.\n\nA...
2,3,DELETE,True,Implement/Test API: DELETE /1.0/kb/accounts/{a...,This API removes tags from a specific account....
3,4,GET,True,Implement/Test API: GET /1.0/kb/accounts/{acco...,This API retrieves custom fields for a specifi...
4,5,POST,True,Implement/Test API: POST /1.0/kb/accounts/{acc...,This API adds custom fields to a specific acco...
...,...,...,...,...,...
195,196,DELETE,False,[API][DELETE /1.0/kb/invoicePayments/{paymentI...,This API endpoint is intended to remove tags f...
196,197,GET,False,[API][GET /1.0/kb/invoicePayments/{paymentId}/...,This API endpoint is intended to retrieve paym...
197,198,POST,False,[API][POST /1.0/kb/invoicePayments/{paymentId}...,This API endpoint is intended to add custom fi...
198,199,PUT,False,[API][PUT /1.0/kb/invoicePayments/{paymentId}/...,This API endpoint is intended to modify custom...


In [5]:
df["Method"].value_counts(), df2["method"].value_counts()

(Method
 GET       50
 POST      21
 DELETE    15
 PUT       14
 Name: count, dtype: int64,
 method
 GET       100
 POST       42
 DELETE     30
 PUT        28
 Name: count, dtype: int64)

In [6]:
(df[:100]["Method"] == df2[:100]["method"]).all()

np.True_

In [7]:
(df[:100]["Title/Summary"] == df2[:100]["title"]).all()

np.True_

In [8]:
# list all files matching dataset/benchmark_feautures/*.feature 
import glob
import os

ticket_ids = []
# Method 3: Using os.listdir with filtering
directory = "dataset/benchmark_feautures"
if os.path.exists(directory):
    all_files = os.listdir(directory)
    feature_files = [f for f in all_files if f.endswith('.feature')]
    for file in feature_files:
        # print(file)
        ticket_ids.append(int(file.split("_")[0]))
else:
    print(f"Directory {directory} does not exist")

for id in range(1, 101):
    if id not in ticket_ids:
        print("Not found:", id)

len(ticket_ids)

100

In [9]:
df["Valid Output?"].value_counts(), df["Gherkin Compliant?"].value_counts(), df["Quality/Coverage?"].value_counts()

(Valid Output?
 YES    100
 Name: count, dtype: int64,
 Gherkin Compliant?
 YES    100
 Name: count, dtype: int64,
 Quality/Coverage?
 9    52
 8    47
 7     1
 Name: count, dtype: int64)

In [10]:
df2 = pd.DataFrame()
df2["jira_id"] = df["Ticket ID"]
df2["coverage_percentage"] = df["Quality/Coverage?"] * 10
df2

,jira_id,coverage_percentage
0,1,90
1,2,80
2,3,80
3,4,90
4,5,80
...,...,...
95,96,90
96,97,80
97,98,90
98,99,90


In [11]:
df2.describe()

,jira_id,coverage_percentage
count,100.000000,100.000000
mean,50.500000,85.100000
std,29.011492,5.221362
min,1.000000,70.000000
25%,25.750000,80.000000
50%,50.500000,90.000000
75%,75.250000,90.000000
max,100.000000,90.000000


In [12]:
df2.to_csv("./dataset/jira_coverage_ground_truth.csv", index=False)